In our article [Solution Revenge: Catching the Mice](catching_the_mice_modular.ipynb) we worked out an involved method of using sums of remainders to find solutions to the "catching the mice problem." The method works for determining if the white mouse is eaten in the first, second, or third rounds (for a population of 13 mice). A follow up question is: would this method work for a four round problem (one larger than Dudeney proposed)?

Let's set up some Python code to look at that.

In [1]:
# imports
import functools
import numpy as np
import pandas as pd
import sympy as sp
from IPython.display import Code, display
from catching_the_mice_fns import (
    run_cat_process,
    WHITE_MOUSE,
)

In [2]:

# for number theory reasons we consider only solutions with different remainders by
# the shared_modulus as truly different solutions
shared_modulus = functools.reduce(sp.lcm, range(1, 14))

shared_modulus

360360

In [3]:


modulo_sum_frame = {
    'advance': np.array(range(shared_modulus), dtype=int),
}
moduli = []
for mi in reversed(range(10, 13 + 1)):
    moduli.append(mi)
    col_name = ' + '.join([f'(x%{m})' for m in moduli])
    col_values = np.zeros(shared_modulus, dtype=int)
    for m in moduli:
        col_values += modulo_sum_frame['advance'] % m
    modulo_sum_frame[col_name] = col_values
for k in range(1, 5):
    modulo_sum_frame[f'is {k} round solution'] = [run_cat_process(start=0, advance=advance, k=k)[k-1] == WHITE_MOUSE for advance in range(shared_modulus)]
modulo_sum_frame = pd.DataFrame(modulo_sum_frame)


modulo_sum_frame

,advance,(x%13),(x%13) + (x%12),(x%13) + (x%12) + (x%11),(x%13) + (x%12) + (x%11) + (x%10),is 1 round solution,is 2 round solution,is 3 round solution,is 4 round solution
0,0,0,0,0,0,True,False,False,False
1,1,1,2,3,4,False,False,False,False
2,2,2,4,6,8,False,False,False,False
3,3,3,6,9,12,False,False,False,False
4,4,4,8,12,16,False,False,False,False
...,...,...,...,...,...,...,...,...,...
360355,360355,8,15,21,26,False,False,False,False
360356,360356,9,17,24,30,False,False,False,False
360357,360357,10,19,27,34,False,False,False,False
360358,360358,11,21,30,38,False,False,False,False


In [4]:
def summarize_results(*, pre_condition, sum_col_name: str, solution_col_name: str):
    interesting_values = sorted(set(modulo_sum_frame.loc[pre_condition & modulo_sum_frame[solution_col_name], sum_col_name]))
    post_condition = pre_condition & (modulo_sum_frame[solution_col_name] == False)
    data_of_interest = pd.DataFrame({
        sum_col_name: modulo_sum_frame.loc[pre_condition, sum_col_name],
        solution_col_name: modulo_sum_frame.loc[pre_condition, solution_col_name],
    })
    non_interesting_positions = np.isin(data_of_interest[sum_col_name], interesting_values) == False
    data_of_interest[sum_col_name] = data_of_interest[sum_col_name].astype(str)
    data_of_interest.loc[non_interesting_positions, sum_col_name] = 'other'
    return (
        pd.crosstab(
            data_of_interest[sum_col_name],
            data_of_interest[solution_col_name],
        ),
        post_condition
    )

We confirm the methodology for a one round problem below.

In [5]:
pre_condition_1 = np.array([True] * shared_modulus, dtype=bool)
ct_1, pre_condition_2 = summarize_results(pre_condition=pre_condition_1, sum_col_name='(x%13)', solution_col_name='is 1 round solution')

ct_1

is 1 round solution,False,True
(x%13),,
0,0,27720
other,332640,0


The above confirms the method because in the above cross tabulation each row is only non-zero in one column. So knowing the sum tells us exactly if we have a solution or not (after the prior conditions are filtered for).

Similarly, we confirm the methodology for a two round problem below.

In [6]:
ct_2, pre_condition_3 = summarize_results(pre_condition=pre_condition_2, sum_col_name='(x%13) + (x%12)', solution_col_name='is 2 round solution')

ct_2

is 2 round solution,False,True
(x%13) + (x%12),,
12,0,27720
other,304920,0


We confirm the methodology for a three round problem below.

In [7]:
ct_3, pre_condition_4 = summarize_results(pre_condition=pre_condition_3, sum_col_name='(x%13) + (x%12) + (x%11)', solution_col_name='is 3 round solution')

ct_3

is 3 round solution,False,True
(x%13) + (x%12) + (x%11),,
11,0,13860
23,0,13860
other,277200,0


And we see the methodology isn't complete for a 4 round problem.

In [8]:
ct_4, pre_condition_5 = summarize_results(pre_condition=pre_condition_4, sum_col_name='(x%13) + (x%12) + (x%11) + (x%10)', solution_col_name='is 4 round solution')

ct_4

is 4 round solution,False,True
(x%13) + (x%12) + (x%11) + (x%10),,
10,0,4620
21,7770,9240
22,7770,9240
33,0,4620
other,233940,0


 We see for `(x%13) + (x%12) + (x%11) + (x%10)` the sums `21` and `22` occur for both solutions and non-solutions. So for these sums we would need more conditions. 
 
 Also, the number of conditions we have to check is growing with the number of problem rounds, so at some point conditioning on the sum may no longer be a greatly advantageous filter.